In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np

In [2]:
imdb_reviews = pd.read_csv("imdb_reviews.csv")
test_reviews = pd.read_csv("test_reviews.csv")

In [4]:
imdb_reviews.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [9]:
word_index = pd.read_csv("word_indexes.csv")

In [10]:
word_index.head(n = 20)

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411
5,spiders,16118
6,hanging,2348
7,woody,2292
8,trawling,52011
9,hold's,52012


In [11]:
word_index = dict(zip(word_index.Words, word_indexes.Indexes)) 

In [21]:
word_index["<PAD>"] = 0
word_index["<START"] = 1
word_index["UNK"] = 2
word_index["UNUSED"] = 3

In [13]:
def review_encoder(text):
    arr = [word_index[word] for word in text]
    return arr

In [14]:
train_data, train_labels = imdb_reviews['Reviews'], imdb_reviews['Sentiment']
test_data, test_labels = test_reviews['Reviews'], test_reviews['Sentiment']

In [18]:
train_data = train_data.apply(lambda review : review.split())
test_data = test_data.apply(lambda review : review.split())

In [22]:
train_data = train_data.apply(review_encoder)
test_data = test_data.apply(review_encoder)

In [24]:
def encode_sentiments(sentiment):
    if sentiment == 'postive':
        return 1
    else:
        return 0


train_labels = train_labels.apply(encode_sentiments)
test_labels = test_labels.apply(encode_sentiments)

In [25]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value = word_index["<PAD>"], padding = 'post', maxlen=500)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value = word_index["<PAD>"], padding = 'post', maxlen = 500)

In [27]:
model = keras.Sequential([keras.layers.Embedding(10000, 16, input_length = 500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16, activation = 'relu'),
                        keras.layers.Dense(1,activation = 'sigmoid')])

In [28]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [29]:
history = model.fit(train_data, train_labels, epochs = 30, batch_size = 512, validation_data = (test_data, test_labels))

Epoch 1/30
49/49 [==============================] - 7s 92ms/step - loss: 0.6379 - accuracy: 0.9133 - val_loss: 0.5375 - val_accuracy: 1.0000
Epoch 2/30
49/49 [==============================] - 4s 73ms/step - loss: 0.4088 - accuracy: 1.0000 - val_loss: 0.2741 - val_accuracy: 1.0000
Epoch 3/30
49/49 [==============================] - 4s 73ms/step - loss: 0.1773 - accuracy: 1.0000 - val_loss: 0.1013 - val_accuracy: 1.0000
Epoch 4/30
49/49 [==============================] - 4s 73ms/step - loss: 0.0663 - accuracy: 1.0000 - val_loss: 0.0412 - val_accuracy: 1.0000
Epoch 5/30
49/49 [==============================] - 4s 74ms/step - loss: 0.0297 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 1.0000
Epoch 6/30
49/49 [==============================] - 4s 73ms/step - loss: 0.0161 - accuracy: 1.0000 - val_loss: 0.0122 - val_accuracy: 1.0000
Epoch 7/30
49/49 [==============================] - 4s 73ms/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.0079 - val_accuracy: 1.0000
Epoch 8/30
49

In [30]:
loss, accuracy = model.evaluate(test_data, test_labels)

782/782 [==============================] - 4s 5ms/step - loss: 1.8231e-04 - accuracy: 1.0000


In [31]:
index = np.random.randint(1,1000)

In [32]:
user_review = test_reviews.loc[index]
print(user_review)

Reviews      <START eric ability to roll from character to ...
Sentiment                                             positive
Name: 23, dtype: object


In [33]:
user_review = test_data[index]
user_review = np.array([user_review])
if(model.predict(user_review) > 0.5).astype("int32"):
    print("Postive Sentiment")
else:
    print("Negative Sentiment")

1/1 [==============================] - 0s 237ms/step
Negative Sentiment
